# Deep Pensieve™
 
#### A Deep Residual Enhanced Attention Maximizing Super Resolution Variational Auto-Encoder (DREAM+SRVAE)

## Multi-Stage Variational Auto-Encoders for Coarse-to-Fine Image Generation 

https://arxiv.org/abs/1705.07202

<img src="https://s3.amazonaws.com/neurokinetikz/download-1.png">

Variational auto-encoder (VAE) is a powerful unsupervised learning framework for image generation. One drawback of VAE is that it generates blurry images due to its Gaussianity assumption and thus L2 loss. To allow the generation of high quality images by VAE, we increase the capacity of decoder network by employing residual blocks and skip connections, which also enable efficient optimization. To overcome the limitation of L2 loss, we propose to generate images in a multi-stage manner from coarse to fine. 

In the simplest case, the proposed multi-stage VAE divides the decoder into two components in which the second component generates refined images based on the course images generated by the first component. Since the second component is independent of the VAE model, it can employ other loss functions beyond the L2 loss and different model architectures. 

The proposed framework can be easily generalized to contain more than two components. Experiment results on the MNIST and CelebA datasets demonstrate that the proposed multi-stage VAE can generate sharper images as compared to those from the original VAE.

## GPU Selection

In [ ]:
MULTI_GPU = True

BATCH_SIZE = 8

# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# # The GPU id to use, usually either "0" or "1";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";

## Imports

In [ ]:
import time
import json
import random
import pickle
import numpy as np
import tensorflow as tf

from libs import utils, gif
from libs.variance_pooling import GlobalVariancePooling2D

from keras.models import Model, load_model
from keras.layers import Input, Flatten, Reshape, Add, Multiply, Lambda
from keras.layers import Dense, Conv2D, SeparableConv2D, MaxPooling2D, Activation
from keras.callbacks import LambdaCallback
from keras.utils import multi_gpu_model

from keras import optimizers
from keras import backend as K

from keras_contrib.losses import DSSIMObjective
from keras_contrib.layers.normalization.groupnormalization import GroupNormalization
from keras_contrib.layers.convolutional.subpixelupscaling import SubPixelUpscaling

## Load Images

In [ ]:
DIRECTORY = 'roadtrip'
SIZE = 64
SCALE_FACTOR = 2

In [ ]:
CHANNELS = 3

FEATURES = SIZE*SIZE*CHANNELS
FEATURES_2X = SCALE_FACTOR*SIZE*SCALE_FACTOR*SIZE*CHANNELS

MODEL_NAME = DIRECTORY+'-'+str(SIZE)+'-'+str(time.time())

In [ ]:
# load images
imgs, xs, ys  = utils.load_images(directory="imgs/"+DIRECTORY,rx=SIZE,ry=SIZE)
imgs_2x, xs_2x, ys_2x = utils.load_images(directory="imgs/"+DIRECTORY,rx=SCALE_FACTOR*SIZE,ry=SCALE_FACTOR*SIZE)

# normalize pixels
IMGS = imgs/127.5 - 1
IMGS_C = IMGS ; IMGS_F = np.reshape(IMGS_C,(-1,FEATURES))

# normalize 2x pixels
IMGS_2X = imgs_2x/127.5 - 1
IMGS_2X_C = IMGS_2X ; IMGS_2X_F = np.reshape(IMGS_2X_C,(-1,FEATURES_2X))

SAMPLES =  np.random.permutation(IMGS_F)[:4]
SAMPLES_2X =  np.random.permutation(IMGS_2X_F)[:4]

TOTAL_BATCH = IMGS.shape[0]

# print shapes
print("MODEL: ",MODEL_NAME)
print("IMGS: ",IMGS_C.shape,IMGS_2X_C.shape)
print("FLAT: ",IMGS_F.shape,IMGS_2X_F.shape)
print("SAMPLES: ",SAMPLES.shape,SAMPLES_2X.shape)

# Encoder

<img src="https://s3.amazonaws.com/neurokinetikz/Asset+4.png">

In [ ]:
def encode(x):
    # set current layer
    current_layer = Reshape((SIZE,SIZE,CHANNELS))(x)
        
    # convolution layers
    for layer, n in enumerate(FILTERS):

        # 3x3 convolution with group normalization + activation
        current_layer = Conv2D(n,3,padding='SAME',kernel_initializer=INITIALIZER)(current_layer)
        current_layer = GroupNormalization(groups=n,axis=-1)(current_layer)
        current_layer = Activation(ACTIVATION)(current_layer)
                
        # dilation rate
        dilation_rate = 1 if (layer+1 == len(FILTERS)) else 2     
        
        # dilated 3x3 convolution with group normalization + activation
        current_layer = Conv2D(n,3,padding='SAME',dilation_rate=dilation_rate,kernel_initializer=INITIALIZER)(current_layer)
        current_layer = GroupNormalization(groups=n,axis=-1)(current_layer)
        current_layer = Activation(ACTIVATION)(current_layer)
                
        # max pooling
        current_layer = MaxPooling2D()(current_layer)
   
    # 1x1 convolution on 4x4 feature maps
    current_layer = Conv2D(int(LATENT_DIM/16),1,padding='SAME',kernel_initializer=INITIALIZER)(current_layer)
        
    # flatten
    flat = Flatten()(current_layer)

    # latent vector
    z = Dense(LATENT_DIM,kernel_initializer=INITIALIZER,activation=ACTIVATION,name='encoder')(flat)
    
    return z

### Very Deep Convolutional Networks for Large-Scale Image Recognition 

https://arxiv.org/abs/1409.1556

<img src="https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/CascadingConvolutions.png">

In this work we investigate the effect of the convolutional network depth on its accuracy in the large-scale image recognition setting. Our main contribution is a thorough evaluation of networks of increasing depth using an architecture with very small (3×3) convolution filters, which shows that a significant improvement on the prior-art configurations can be achieved by pushing the depth to 16–19 weight layers.

First, we incorporate three non-linear rectification layers instead of a single one, which makes the decision function more discriminative.

Second, we decrease the number of parameters: assuming that both the input and the output of a
three-layer 3 × 3 convolution stack has C channels, the stack is parametrised by (W) weights; at the same time, a single 7 × 7 conv. layer would require 81% more. This can be seen as imposing a regularisation on the 7 × 7 conv. filters, forcing them to have a decomposition through the 3 × 3 filters (with non-linearity injected in between)

### Dilated Residual Networks

https://arxiv.org/abs/1705.09914v1

<img width="600" src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2019-01-22+at+10.06.28+AM.png">

Convolutional networks for image classification progressively reduce resolution until the image is represented by tiny feature maps in which the spatial structure of the scene is no longer discernible. Such loss of spatial acuity can limit image classification accuracy and complicate the transfer of the model to downstream applications that require detailed scene understanding. These problems can be alleviated by dilation, which increases the resolution of output feature maps without reducing the receptive field of individual neurons. We show that dilated residual networks (DRNs) outperform their non-dilated counterparts in image classification without increasing the model’s depth or complexity

### Receptive Field Calculator 

https://fomoro.com/projects/project/receptive-field-calculator

<img width="600" src="https://s3.amazonaws.com/neurokinetikz/download.png">

### Group Normalization

https://medium.com/syncedreview/facebook-ai-proposes-group-normalization-alternative-to-batch-normalization-fb0699bffae7

<img src="https://s3.amazonaws.com/neurokinetikz/groupnorm.png">


The mainstream normalization technique for almost all convolutional neural networks today is Batch Normalization (BN), which has been widely adopted in the development of deep learning. Proposed by Google in 2015, BN can not only accelerate a model’s converging speed, but also alleviate problems such as Gradient Dispersion in the deep neural network, making it easier to train models.

Dr. Wu and Dr. He however argue in their paper Group Normalization that normalizing with batch size has limitations, as BN cannot ensure the model accuracy rate when the batch size becomes smaller. As a result, researchers today are normalizing with large batches, which is very memory intensive, and are avoiding using limited memory to explore higher-capacity models.

Dr. Wu and Dr. He believe their new GN technique is a simple but effective alternative to BN. Specifically, GN divides channels — also referred to as feature maps that look like 3D chunks of data — into groups and normalizes the features within each group. GN only exploits the layer dimensions, and its computation is independent of batch sizes.

The paper reports that GN had a 10.6% lower error rate than its BN counterpart for ResNet-50 in ImageNet with a batch size of 2 samples; and matched BN performance while outperforming other normalization techniques with a regular batch size.

## InfoVAE: Information Maximizing Variational Autoencoders

In [ ]:
def compute_kernel(x, y):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

In [ ]:
def vae_loss(y_true,y_pred):
    epsilon = tf.random_normal(tf.stack([BATCH_SIZE, LATENT_DIM]))
    latent_loss = compute_mmd(epsilon, y_pred)
    return latent_loss

https://arxiv.org/abs/1706.02262v3

https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/

<table><tr>
<td><img src="https://s3.amazonaws.com/neurokinetikz/kl_latent.gif" ></td>
<td><img src="https://s3.amazonaws.com/neurokinetikz/mmd_latent.gif" ></td>
</tr></table>

Maximum mean discrepancy (MMD, (Gretton et al. 2007)) is based on the idea that two distributions are identical if and only if all their moments are the same. Therefore, we can define a divergence by measuring how “different” the moments of two distributions p(z) and q(z) are. MMD can accomplish this efficiently via the kernel embedding trick:

A kernel can be intuitively interpreted as a function that measures the “similarity” of two samples. It has a large value when two samples are similar, and small when they are different. For example, the Gaussian kernel considers points that are close in Euclidean space to be “similar”. A rough intuition of MMD, then, is that if two distributions are identical, then the average “similarity” between samples from each distribution, should be identical to the average “similarity” between mixed samples from both distributions.

# Decoder

<img src="https://s3.amazonaws.com/neurokinetikz/Asset+5.png">

<br>

In [ ]:
def decode(z,z_g):
    # current layer
    current_layer = z ; generator = z_g
    
    # reshape
    reshape = Reshape((4,4,int(LATENT_DIM/16)))
    current_layer = reshape(current_layer) ; generator = reshape(generator)
    
    # 1x1 convolution 
    c0 = Conv2D(int(LATENT_DIM/16),1,padding="SAME",kernel_initializer=INITIALIZER)
    current_layer = c0(current_layer) ; generator = c0(generator)
        
    # reverse the filters
    filters = FILTERS[::-1]
    
    # build layers
    for layer, n in enumerate(filters):

        # upsample
        u = SubPixelUpscaling(scale_factor=2)
        current_layer = u(current_layer) ; generator = u(generator)

        # dilation rate
        dilation_rate = 1 if (layer == 0) else 2
            
        # dilated 3x3 convolution with group normalization + activation
        c1 = Conv2D(n,3,padding='SAME',dilation_rate=dilation_rate,kernel_initializer=INITIALIZER)
        g1 = GroupNormalization(groups=n,axis=-1)
        a1 = Activation(ACTIVATION)
        
        current_layer = c1(current_layer) ; generator = c1(generator)
        current_layer = g1(current_layer) ; generator = g1(generator)
        current_layer = a1(current_layer) ; generator = a1(generator)
        
        # 3x3 convolution with group normalization + activation
        c2 = Conv2D(n,3,padding='SAME',kernel_initializer=INITIALIZER)
        g2 = GroupNormalization(groups=n,axis=-1)
        a2 = Activation(ACTIVATION)
        
        current_layer = c2(current_layer) ; generator = c2(generator)
        current_layer = g2(current_layer) ; generator = g2(generator)
        current_layer = a2(current_layer) ; generator = a2(generator)
            
    # output convolution
    c = Conv2D(CHANNELS,1,padding='SAME', activation='tanh',name='decoder_dssim')
    current_layer = c(current_layer) ; generator = c(generator)
    
    # flatten
    flatten = Flatten(name='decoder')
    decoder_loss = flatten(current_layer)
    
    return current_layer, generator, decoder_loss

## Deconvolution and Checkerboard Artifacts 

https://distill.pub/2016/deconv-checkerboard/

When we have neural networks generate images, we often have them build them up from low resolution, high-level descriptions. This allows the network to describe the rough image and then fill in the details.

In order to do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. Roughly, deconvolution layers allow the model to use every point in the small image to “paint” a square in the larger one.

Unfortunately, deconvolution can easily have “uneven overlap,” putting more of the metaphorical paint in some places than others. In particular, deconvolution has uneven overlap when the kernel size (the output window size) is not divisible by the stride (the spacing between points on the top). While the network could, in principle, carefully learn weights to avoid this  — as we’ll discuss in more detail later — in practice neural networks struggle to avoid it completely.

<img src="https://s3.amazonaws.com/neurokinetikz/download-2.png">

To avoid these artifacts, we’d like an alternative to regular deconvolution (“transposed convolution”). Unlike deconvolution, this approach to upsampling shouldn’t have artifacts as its default behavior. Ideally, it would go further, and be biased against such artifacts.

One approach is to separate out upsampling to a higher resolution from convolution to compute features. For example, you might resize the image (using nearest-neighbor interpolation or bilinear interpolation) and then do a convolutional layer. This seems like a natural approach, and roughly similar methods have worked well in image super-resolution.

Our experience has been that nearest-neighbor resize followed by a convolution works very well, in a wide variety of contexts.

# Refiner

<img  src="https://s3.amazonaws.com/neurokinetikz/Asset%2B10-100.jpg">

In [ ]:
def refine(x,x_g,name='refiner'):
    # 1x1 channel convolution
    c1 = Conv2D(CHANNELS,1,padding='SAME')
    current_layer = c1(x) ; generator = c1(x_g)
    
    # shortcut
    shortcut = current_layer; shortcut_g = generator
    
    # reshape convolution
    c2 = Conv2D(R_FILTERS,3,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c2(x) ; generator = c2(x_g)
    
    # residual groups
    for i in range(R_GROUPS):
        current_layer, generator = residual_group(current_layer,generator)
    
    # output convolution
    c3 = Conv2D(CHANNELS,1,padding='SAME')
    current_layer = c3(current_layer); generator = c3(generator)
    
    # merge shortcut
    merge = Add()
    current_layer = merge([current_layer, shortcut]) ; generator = merge([generator, shortcut_g])
    
    # output activation
    activation = Activation('tanh',name=name+'_dssim')
    current_layer = activation(current_layer) ; generator = activation(generator)
    
    # flatten
    flatten = Flatten(name=name)
    refiner_loss = flatten(current_layer)
    
    return current_layer, generator, refiner_loss

In [ ]:
def residual_group(x,x_g):
    
    # current layer
    current_layer = x ; generator = x_g
    
    # shortcut
    shortcut = current_layer ; shortcut_g = generator
    
    # residual blocks
    for i in range(R_BLOCKS):
        
        # residual + attention
        current_layer, generator = residual(current_layer, generator, R_ATTENTION)
    
    # merge shortcut
    merge = Add()
    current_layer = merge([current_layer, shortcut]) ; generator = merge([generator, shortcut_g])
    
    # scale
    scale = Lambda(lambda x: x * .01)
    current_layer = scale(current_layer) ; generator = scale(generator)
    
    # convolution
    conv = Conv2D(R_FILTERS,3,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = conv(current_layer) ; generator = conv(generator)
    
    return current_layer, generator

## Enhanced Deep Residual Networks for Single Image Super-Resolution (EDSR)

https://arxiv.org/abs/1707.02921

<img width=500 src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2018-11-18+at+8.54.43+AM.png">

Recently, the powerful capability of deep neural networks has led to dramatic improvements in SR. Since Dong et al. [4, 5] first proposed a deep learning-based SR method, various CNN architectures have been studied for SR. Kim et al. [11, 12] first introduced the residual network for training much deeper network architectures and achieved superior performance. In particular, they showed that skip connection and recursive convolution alleviate the burden of carrying identity information in the super-resolution network. Similarly to [20], Mao et al. [16] tackled the general image restoration problem with encoder-decoder networks and symmetric skip connections. In [16], they argue that those nested skip connections provide fast and improved convergence.

<img src="https://s3.amazonaws.com/neurokinetikz/download-3.png">


We remove the batch normalization layers from our network as Nah et al.[19] presented in their image deblurring work. Since batch normalization layers normalize the features, they get rid of range flexibility from networks by normalizing the features, it is better to remove them. We experimentally show that this simple modification increases the performance substantially as detailed in

Furthermore, GPU memory usage is also sufficiently reduced since the batch normalization layers consume the same amount of memory as the preceding convolutional layers. Our baseline model without batch normalization layer saves approximately 40% of memory usage during training, compared to SRResNet. Consequently, we can build up a larger model that has better performance than conventional ResNet structure under limited computational resources.

In [ ]:
def residual(x,x_g,attention=False):
    # current layer
    current_layer = x ; generator = x_g

    # shortcuts
    shortcut = current_layer ; shortcut_g = generator

    # separable convolution
    c1 = SeparableConv2D(R_FILTERS,R_SEPARABLE,padding='SAME',activation=ACTIVATION,kernel_initializer=INITIALIZER)
    current_layer = c1(current_layer) ; generator = c1(generator)

    # separable convolution
    c2 = SeparableConv2D(R_FILTERS,R_SEPARABLE,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c2(current_layer) ; generator = c2(generator)
    
    # residual scaling
    scale = Lambda(lambda x: x * .01)
    current_layer = scale(current_layer) ; generator = scale(generator)
    
    # residual attention
    if(attention):
        current_layer, generator = residual_attention(current_layer,generator)
    
    # merge shortcut
    merge = Add()
    current_layer = merge([current_layer, shortcut]) ; generator = merge([generator, shortcut_g])

    return current_layer, generator

## Residual Bottleneck Attention Module (RBAM)

<img src="https://s3.amazonaws.com/neurokinetikz/Asset+8.png">

In [ ]:
def residual_attention(x,x_g):
    # current layer
    current_layer = x ; generator = x_g
    
    # shortcuts
    shortcut = current_layer ; shortcut_g = generator
    
    # channel attention
    ca, ca_g = channel_attention(current_layer, generator)
    
    # spatial attention
    sa, sa_g = spatial_attention(current_layer, generator)
    
    # fuse channel and spatial attention
    fuse = Add()
    current_layer = fuse([ca,sa]); generator = fuse([ca_g,sa_g])
    
    # sigmoid activation
    s = Activation("sigmoid")
    current_layer = s(current_layer) ; generator = s(generator)
    
    # merge fused attention with shortcut
    m = Multiply()
    current_layer = m([current_layer,shortcut]) ; generator = m([generator, shortcut_g])
    
    return current_layer, generator

## Image Super-Resolution Using Very Deep Residual Channel Attention Networks (RCAB)

https://arxiv.org/abs/1807.02758v2

<img width="600" src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2018-12-16+at+11.12.08+AM.png">

To make a further step, we propose channel attention (CA) mechanism to adaptively rescale each channel-wise feature by modeling the interdependencies across feature channels. Such CA mechanism allows our proposed network to concentrate on more useful channels and enhance discriminative learning ability.

## Residual Attention Module for Single Image Super-Resolution (RAM)

https://arxiv.org/abs/1811.12043

<img width="500" src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2018-12-07+at+8.51.39+AM.png">

In this paper, we propose a new attention method, which is composed of new channel-wise and spatial attention mechanisms optimized for SR and a new fused attention to combine them. Based on this, we propose a new residual attention module (RAM) and a SR network using RAM (SRRAM). We provide in-depth experimental analysis of different attention mechanisms in SR. It is shown that the proposed method can construct both deep and lightweight SR networks showing improved performance in comparison to existing state-of-the-art methods.

In [ ]:
def channel_attention(x,x_g):
    # current layer
    current_layer = x ; generator = x_g
    
    # global average/variance pooling
    gvp = GlobalVariancePooling2D()
    
    reshape = None
    reshape = Reshape((1,1,R_FILTERS))
        
    current_layer = reshape(gvp(current_layer)); generator = reshape(gvp(generator))
    
    # squeeze & excitation
    squeeze = Conv2D(int(R_FILTERS/R_REDUCTION),1,padding='SAME',activation=ACTIVATION,kernel_initializer=INITIALIZER)
    current_layer = squeeze(current_layer); generator = squeeze(generator);
    
    # scaling
    c2 = Conv2D(R_FILTERS,1,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c2(current_layer) ; generator = c2(generator)
        
    return current_layer, generator

## BAM: Bottleneck Attention Module 

https://arxiv.org/abs/1807.06514v2

<img width="600" src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2018-12-16+at+10.15.09+AM.png">

In this work, we focus on the effect of attention in general deep neural networks. We propose a simple and effective attention module, named Bottleneck Attention Module (BAM), that can be integrated with any feed-forward convolutional neural networks. Our module infers an attention map along two separate pathways, channel and spatial. We place our module at each bottleneck of models where the downsampling of feature maps occurs. Our module constructs a hierarchical attention at bottlenecks with a number of parameters and it is trainable in an end-to-end manner jointly with any feed-forward models.

As the channels of feature maps can be regarded as feature detectors, the two branches (spatial and channel) explicitly learn ‘what’ and ‘where’ to focus on.

In [ ]:
def spatial_attention(x,x_g):
    # current layer
    current_layer = x ; generator = x_g
    
    # 1x1 convolution
    c1 = Conv2D(int(R_FILTERS/R_REDUCTION),1,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c1(current_layer); generator = c1(generator)

    # dilated convolution
    c2 = Conv2D(int(R_FILTERS/R_REDUCTION),3,dilation_rate=R_DILATION,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c2(current_layer) ; generator = c2(generator)

    # dilated convolution
    c3 = Conv2D(int(R_FILTERS/R_REDUCTION),3,dilation_rate=R_DILATION,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c3(current_layer) ; generator = c3(generator)

    # 1x1 convolution
    c4 = Conv2D(1,1,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c4(current_layer); generator = c4(generator)
    
    return current_layer, generator

## Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network 

https://arxiv.org/abs/1609.05158

<img width="75%" src="https://s3.amazonaws.com/neurokinetikz/Screen+Shot+2018-11-23+at+10.26.45+AM.png">

Recently, several models based on deep neural networks have achieved great success in terms of both reconstruction accuracy and computational performance for single image super-resolution. In these methods, the low resolution (LR) input image is upscaled to the high resolution (HR) space using a single filter, commonly bicubic interpolation, before reconstruction. This means that the super-resolution (SR) operation is performed in HR space. We demonstrate that this is sub-optimal and adds computational complexity. 

In this paper, we present the first convolutional neural network (CNN) capable of real-time SR of 1080p videos on a single K2 GPU. To achieve this, we propose a novel CNN architecture where the feature maps are extracted in the LR space. In addition, we introduce an efficient sub-pixel convolution layer which learns an array of upscaling filters to upscale the final LR feature maps into the HR output. By doing so, we effectively replace the handcrafted bicubic filter in the SR pipeline with more complex upscaling filters specifically trained for each feature map, whilst also reducing the computational complexity of the overall SR operation. 

We evaluate the proposed approach using images and videos from publicly available datasets and show that it performs significantly better (+0.15dB on Images and +0.39dB on Videos) and is an order of magnitude faster than previous CNN-based methods.

In [ ]:
def upsample(x,x_g,img_g,name='super'):
    
    # current layer
    current_layer = x ; generator = x_g; img_generator = img_g
    
    # convolution
    c1 = Conv2D(R_FILTERS*SCALE_FACTOR*SCALE_FACTOR,3,padding='SAME',activation=ACTIVATION,kernel_initializer=INITIALIZER)
    current_layer = c1(current_layer) ; generator = c1(generator) ; img_generator = c1(img_generator)
    
    # sub-pixel upscaling
    upscale = SubPixelUpscaling(scale_factor=SCALE_FACTOR)
    current_layer = upscale(current_layer); generator = upscale(generator); img_generator = upscale(img_generator)
    
    # In practice, it is useful to have a second convolution layer after the 
    # SubPixelUpscaling layer to speed up the learning process.
    c2 = Conv2D(R_FILTERS*SCALE_FACTOR*SCALE_FACTOR,3,padding='SAME',activation=ACTIVATION,kernel_initializer=INITIALIZER)
    current_layer = c2(current_layer) ; generator = c2(generator); img_generator = c2(img_generator)
    
    # output convolution
    c3 = Conv2D(CHANNELS,1,padding='SAME',activation='tanh',name=name+"_dssim")
    current_layer = c3(current_layer); generator = c3(generator); img_generator = c3(img_generator)
    
    # flatten
    flatten = Flatten(name=name)
    upscale_loss = flatten(current_layer)
    
    return current_layer, generator, img_generator, upscale_loss

# Model

## Fast and Accurate Deep Network Learning by Exponential Linear Units (ELUs) 

https://arxiv.org/abs/1511.07289

<img width="300" style="float:left;" src="https://blogs.mathworks.com/deep-learning/files/2017/12/defining_elu_layer_01.png">

We introduce the "exponential linear unit" (ELU) which speeds up learning in deep neural networks and leads to higher classification accuracies. Like rectified linear units (ReLUs), leaky ReLUs (LReLUs) and parametrized ReLUs (PReLUs), ELUs alleviate the vanishing gradient problem via the identity for positive values. However, ELUs have improved learning characteristics compared to the units with other activation functions. 

In contrast to ReLUs, ELUs have negative values which allows them to push mean unit activations closer to zero like batch normalization but with lower computational complexity. Mean shifts toward zero speed up learning by bringing the normal gradient closer to the unit natural gradient because of a reduced bias shift effect. While LReLUs and PReLUs have negative values, too, they do not ensure a noise-robust deactivation state. ELUs saturate to a negative value with smaller inputs and thereby decrease the forward propagated variation and information. 

Therefore, ELUs code the degree of presence of particular phenomena in the input, while they do not quantitatively model the degree of their absence. In experiments, ELUs lead not only to faster learning, but also to significantly better generalization performance than ReLUs and LReLUs on networks with more than 5 layers. 

In [ ]:
# default activation
ACTIVATION  = 'elu'

## Hyper-parameters in Action! Part II — Weight Initializers 

https://towardsdatascience.com/hyper-parameters-in-action-part-ii-weight-initializers-35aee1a28404

<img width="300" style="float:left;" src="https://cdn-images-1.medium.com/max/1600/1*WLUL_bcjsNK9sXNw6nC-cg.png">

If you dug a little bit deeper, you’ve likely also found out that one should use Xavier / Glorot initialization if the activation function is a Tanh, and that He initialization is the recommended one if the activation function is a ReLU.

In summary, for a ReLU activated network, the He initialization scheme using an Uniform distribution is a pretty good choice.



In [ ]:
# initializers
INITIALIZER = 'he_uniform'

## Learning to Generate Images with Perceptual Similarity Metrics

https://arxiv.org/abs/1511.06409

<img width="200" style="float:left;" src="https://s3.amazonaws.com/neurokinetikz/download-4.png">

In this paper, we explore loss functions that, unlike MSE, MAE, and likelihoods, are grounded in human perceptual judgments. We show that these perceptual losses lead to representations are superior to other methods, both with respect to reconstructing given images, and generating novel ones. This superiority is demonstrated both in quantitative studies and human judgements ... We (also) demonstrate that perceptual losses yield a convincing win when applied to a state-of-the-art architecture for single image super-resolution.

As observed in the deterministic case, MS-SSIM is better at capturing fine details than either MSE or MAE.

## Callbacks

In [ ]:
def gifit(epoch=None):
    if (epoch % GIF_STEPS == 0):
        print('saving gif ...')
        zr,yr,yfr,imgr,imgfr,imgsr,imgsfr = MODEL.predict_on_batch(SAMPLES)
        recon = np.clip(127.5*(imgsr+1).reshape((-1,SCALE_FACTOR*SIZE,SCALE_FACTOR*SIZE,CHANNELS)), 0, 255)
        RECONS.append(utils.montage(recon).astype(np.uint8))
        
def saveit(epoch=None):
    if ((epoch > 0) and (epoch % MODEL_STEPS == 0)):
        print('saving model ...')
        if(MULTI_GPU):
            AUTOENCODER.set_weights(PARALLEL.layers[3].get_weights())
        AUTOENCODER.save(MODEL_NAME+'-autoencoder-model.h5')
        ENCODER.save(MODEL_NAME+'-encoder-model.h5')
        DECODER.save(MODEL_NAME+'-generator-model.h5')
        SUPER.save(MODEL_NAME+'-super-model.h5')
        SUPERSIZER.save(MODEL_NAME+'-supersizer-model.h5')
        print('done')
       
        
# callbacks
giffer = LambdaCallback(on_epoch_end=lambda epoch, logs: gifit(epoch))
saver = LambdaCallback(on_epoch_end=lambda epoch, logs: saveit(epoch))

## Model Architecture 

In [ ]:
# Encoder/Decoder
if (SIZE == 256):
    FILTERS = [64,128,192,256,320,384]
    
elif (SIZE == 128):
    FILTERS = [64,128,192,256,320]
    
elif (SIZE == 64):
    FILTERS = [64,128,192,256]
    
elif (SIZE == 32):
    FILTERS = [64,96,128]
    
# Residuals
R_LAYERS  = 16
R_GROUPS = 4
R_BLOCKS = 4
R_FILTERS = 64
R_SEPARABLE = 11
R_SCALING = 0.01

# Attention Modules
R_ATTENTION = True
R_REDUCTION = 4
R_DILATION = 4

# Latent dimension size
LATENT_DIM = 512

## Model Definition 

In [ ]:
# input
X = Input(shape=(FEATURES,))

# encode
Z = encode(X)

# decoder input
Z_G = Input(shape=(LATENT_DIM,))

# decode
Y, Y_G, Y_F = decode(Z,Z_G)

# refine
IMG, IMG_G, IMG_F = refine(Y,Y_G)

# supersizer input
X_G = Input(shape=(SIZE,SIZE,CHANNELS))

# supersize
IMG_S, IMG_G_S, X_G_S, IMG_S_F = upsample(IMG,IMG_G,X_G)

# model definitions
AUTOENCODER = Model(inputs=[X], outputs=[Z,Y,Y_F,IMG,IMG_F,IMG_S,IMG_S_F])
ENCODER = Model(inputs=[X], outputs=[Z])
DECODER = Model(inputs=[Z_G], outputs=[IMG_G])
SUPER = Model(inputs=[Z_G], outputs=[IMG_G_S])
SUPERSIZER = Model(inputs=[X_G], outputs=[X_G_S])

# define optimizer
ADAM = optimizers.Adam(amsgrad=True)

## Compile Model 

In [ ]:
# compile models
ENCODER.compile(optimizer=ADAM,loss='mse')
DECODER.compile(optimizer=ADAM,loss='mae')
SUPER.compile(optimizer=ADAM,loss='mse')
SUPERSIZER.compile(optimizer=ADAM,loss='mse')

# compile model
AUTOENCODER.compile(optimizer=ADAM,loss=LOSSES)

# compile multi-gpu model
if(MULTI_GPU):
    # define autoencoder
    PARALLEL = multi_gpu_model(AUTOENCODER,cpu_relocation=True,gpus=2)
    PARALLEL.compile(optimizer=ADAM,loss=LOSSES)
    PARALLEL.summary()

# print summary
AUTOENCODER.summary()

## Training

In [ ]:
RECONS = []

SCALE_FACTOR= 2

START_EPOCH = 11
EPOCHS      = 101

GIF_STEPS   = 1
MODEL_STEPS = 1000

# fit model
MODEL = PARALLEL if(MULTI_GPU) else AUTOENCODER
MODEL.fit(x=IMGS_F,
            y=[IMGS_F,IMGS_C,IMGS_F,IMGS_C,IMGS_F,IMGS_2X_C,IMGS_2X_F],
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=[giffer,saver],
            initial_epoch=START_EPOCH)

# save training gif
# gif.build_gif(RECONS, saveto=MODEL_NAME+'-final'+ "-"+str(time.time())+'.gif')

print("done")

## Re-animation

In [ ]:
if(MULTI_GPU):
    AUTOENCODER.set_weights(PARALLEL.layers[3].get_weights())

def random_latents(n_imgs=3,steps=30):
    rimgs = np.random.permutation(IMGS_F)[:n_imgs]
    rimgs = np.append(rimgs, [rimgs[0]],axis=0)
    latent_animation(rimgs,steps,filename=str(time.time()))

def latent_animation(imgs=None,steps=None,filename="latent-animation"):
    animate(generate(get_latents(imgs,steps),filename),filename)
    
def get_latents(imgs,steps):
    # get latent encodings for images
    print('encoding latent vectors ...')
    latents = []
    
    for index,img in enumerate(imgs):
        img = np.reshape(img,(-1,FEATURES))
        latent = ENCODER.predict_on_batch(img)
        latents.append(latent)
    
    # calculate latent path
    print('calculating latent path ...')
    latent_path = []
    for i in range(len(latents)-1):
        # get latent vectors
        l1 = latents[i] ; l2 = latents[i+1]

        # calculate latent distance
        image_distance = l2 - l1

        # create the latent path
        for j in range(steps):
            latent_path.append(l1 + j*image_distance/steps)
        latent_path.append(l2)
    
    return latent_path
       
    
def generate(latent_path,filename=None):
     # reconstruct images along the path
    latent_path = np.reshape(latent_path,(-1,LATENT_DIM))
    
    print('decoding ...')
    recons = DECODER.predict_on_batch(latent_path)
    
    if(filename != None):
        print('saving decoder gif')
        build_gif(np.asarray(recons),SIZE,filename)
    
    return recons
    
def animate(recons,filename=None):
    print('supersizing ...')
    chunks = SUPERSIZER.predict_on_batch(recons[:10])
    
    for i in range(10,len(recons),10):
        s2 = SUPERSIZER.predict_on_batch(recons[i:i+10])
        chunks = np.concatenate([chunks,s2])
    
#     print('saving supersizer gif')
#     build_gif(chunks,SIZE*SCALE_FACTOR,filename+"-"+str(SCALE_FACTOR)+"x")
   
#     print(filename)
    return chunks
    
def build_gif(recons,size,filename='latent-animation'):
    final = np.clip((127.5*(recons+1)),0,255)
    gif.build_gif([utils.montage([r]).astype(np.uint8) for r in final], saveto=filename+".gif",dpi=72)

### Grid

In [ ]:
SCALE_FACTOR=2
GRID = 4
N_IMGS = 3
STEPS = 35

for j in range(5):
    grid=[]
    for i in range(GRID):
        imgs = np.random.permutation(IMGS_F)[:N_IMGS]
        imgs = np.append(imgs, [imgs[0]],axis=0)
        latent_path = get_latents(imgs,STEPS)
        recons = animate(generate(latent_path))
        final = np.clip((127.5*(recons+1)).reshape((-1,SIZE*SCALE_FACTOR,SIZE*SCALE_FACTOR,CHANNELS)),0,255)
        grid.append(final)
        
    rs = []
    for img in grid:
        rs.append([frame for frame in img])
    rs = np.moveaxis(grid,1,0)
    print('saving ...')
    filename = str(time.time())+".gif"
    gif.build_gif([utils.montage(r).astype(np.uint8) for r in rs], saveto=filename,dpi=72)

    print('done ... ',filename)

### Sequence 

In [ ]:
SCALE_FACTOR=2
LATENT_DIM=512
for i in range(5):
    random_latents(3,35)

## Load Model

In [ ]:
model_name = "alp-256-1554116194.493014"

print('loading model ...')
AEC = load_model(model_name+"-autoencoder-model.h5",
                 custom_objects={'vae_loss': vae_loss,
                                 'R_SCALING':0.01, 
                                 'DSSIMObjective':DSSIMObjective(),
                                 'GlobalVariancePooling2D': GlobalVariancePooling2D,
                                 'SubPixelUpscaling': SubPixelUpscaling})
print('done')

## Continue Training 